# Analysis: Effects of DETECT on Reporting of EA to APS
2019-04-02

In [ ]:
* Suppress log output;
options nosource nonotes;

# Import data

In [ ]:
proc import 
    datafile = "U:\Research\Detect\one_year_data\aps_ddd.xlsx"
    out      = aps_pilot_data
    dbms     = xlsx
    replace;
run;

# Data management

In [ ]:
* Make sure the ems dummy variable is correct and rename study_week week;
data full_data;
    set aps_pilot_data;
    ems = 0;
    if reporter = "health care providers/staff -- ems/emt" then ems = 1;
    week = study_week;
run;

In [ ]:
* Create a variable that counts the number of reports by week, ems reporter, and service area;
* Drop unneeded variables;
proc sql;
    create table aps_pilot_data as
        select 
            count(study_week) as report_count, /* n reports by week, ems reporter, and service area */
            ems as ems,
            medstar_service_area as medstar_service_area,
            study_week as study_week
            from full_data
            group by study_week, ems, medstar_service_area;
quit;

In [ ]:
* Create a new data set with one row for each possible combination of week, ems reporter, and service area;
* We need to do this to fill in weeks with zero reports in the cell below;
data template;
    do ems = 0 to 1;
        do medstar_service_area = 0 to 1;
            do study_week = 1 to 165;
                output;
            end;
        end;
    end;
run;

In [ ]:
* Sort datasets for merge;
proc sort data = aps_pilot_data; 
    by ems medstar_service_area study_week;
run;

proc sort data = template; 
    by ems medstar_service_area study_week;
run;

In [ ]:
* Merge the weekly report count data with the template data created above;
* This is the final dataset that will be used for analysis;
* Also, add back in detect dummy variable and study_period variable;
data analyze_ddd;
    merge aps_pilot_data template;
    by ems medstar_service_area study_week;

    if report_count = . then report_count = 0;

    if study_week >= 38 and study_week <= 43 then detect = 1;
    else if study_week >= 109 then detect = 1;
    else detect = 0;
    
    if study_week < 38 then study_period = 1;
    else if 38 <= study_week <= 43 then study_period = 2;
    else if 44 <= study_week <= 108 then study_period = 3;
    else study_period = 4;
    
    week = study_week;
run;

# Descriptive statistics

## Percent of all reports made by EMS

In [8]:
proc freq data = full_data;
    table ems;
run;

## Percent of all reports where evidence of some type of abuse was found during APS investigation
For this analysis, it really only makes sense to include reports where an actual investigation was conducted. In other words, drop rows that were administrative closures, unable to locate, etc.

In [9]:
* Percentage valid for EMS in medstar_service area during and not during detect;
proc genmod data = full_data;
    where total_allegations_investigated > 0 & ems = 1 & medstar_service_area = 1;
    class case;
    model any_valid = detect / link = id;
    repeated subject = case / type = CS;
    
    estimate "Proportion valid when detect = 0" INTERCEPT 1;
    estimate "Proportion valid when detect = 1" INTERCEPT 1 detect 1;
    
    ods select GEEEmpPEst Estimates;
run;

## Mean (unadjusted) number of reports in each time period

In [10]:
proc genmod data = analyze_ddd;
    where ems = 1 & medstar_service_area = 1;
    class study_week study_period;
    model report_count = study_period;
    repeated subject = study_week / type = CS;
    lsmeans study_period / cl;
    ods select GEEEmpPEst LSMeans;
run;

# DDD for change in reporting

In [11]:
proc genmod data = analyze_ddd;
    class study_week;
    model report_count = detect ems medstar_service_area 
    detect*ems detect*medstar_service_area medstar_service_area*ems detect*medstar_service_area*ems / type3 link = log dist = nb;
    repeated subject = study_week / type = CS;
    estimate "detect effect among ems=1 and service=1" detect 1 detect*ems 1 detect*medstar_service_area 1 detect*medstar_service_area*ems 1 / exp;
    estimate "detect effect among ems=0 and service=1" detect 1  detect*medstar_service_area 1 / exp;
    estimate "detect effect among ems=1 and service=0" detect 1  detect*ems 1 / exp;
    estimate "detect effect among ems=0 and service=0" detect 1 / exp;
    estimate "DDD as causal effect" detect*medstar_service_area*ems 1 / exp;
run;

# DDD validation

In [12]:
proc genmod data = full_data;
    class study_week any_valid (param = ref ref = FIRST);
    model any_valid = detect ems medstar_service_area detect*ems detect*medstar_service_area medstar_service_area*ems 
    detect*medstar_service_area*ems / link = logit dist = bin;
    repeated subject = study_week / type = CS;

    estimate "detect among ems=1 and service=1" detect 1 detect*ems 1 detect*medstar_service_area 1 detect*medstar_service_area*ems 1 / exp;
    estimate "detect among ems=0 and service=1" detect 1  detect*medstar_service_area 1 / exp;
    estimate "detect among ems=1 and service=0" detect 1  detect*ems 1 / exp;
    estimate "detect among ems=0 and service=0" detect 1 / exp;
    estimate "DDD as causal effect" detect*medstar_service_area*ems 1 / exp;
run;

# Sensitivity analysis with group specific time trends
DDD for change in reporting

In [13]:
proc genmod data = analyze_ddd;
    class study_week;
    model report_count = detect ems medstar_service_area detect*ems detect*medstar_service_area medstar_service_area*ems 
    detect*medstar_service_area*ems week week*ems week*medstar_service_area week*ems*medstar_service_area / type3 link = log dist = nb;
    repeated subject=study_week / type = CS;
    estimate "detect effect among ems=1 and service=1" detect 1 detect*ems 1 detect*medstar_service_area 1 detect*medstar_service_area*ems 1 / exp;
    estimate "detect effect among ems=0 and service=1" detect 1  detect*medstar_service_area 1 / exp;
    estimate "detect effect among ems=1 and service=0" detect 1  detect*ems 1 / exp;
    estimate "detect effect among ems=0 and service=0" detect 1 / exp;
    estimate "DDD as causal effect" detect*medstar_service_area*ems 1 / exp;
run;

DDD validation

In [14]:
proc genmod data = full_data;
    class study_week any_valid(param = ref ref = FIRST);
    model any_valid = detect ems medstar_service_area detect*ems detect*medstar_service_area medstar_service_area*ems 
    detect*medstar_service_area*ems week week*ems week*medstar_service_area week*ems*medstar_service_area / link = logit dist = bin;
    repeated subject = study_week / type = CS;

    estimate "detect among ems=1 and service=1" detect 1 detect*ems 1 detect*medstar_service_area 1 detect*medstar_service_area*ems 1 / exp;
    estimate "detect among ems=0 and service=1" detect 1  detect*medstar_service_area 1 / exp;
    estimate "detect among ems=1 and service=0" detect 1  detect*ems 1 / exp;
    estimate "detect among ems=0 and service=0" detect 1 / exp;
    estimate "DDD as causal effect" detect*medstar_service_area*ems 1 / exp;
run;